# LSTM text generation

Example script to generate text from text files.

At least 20 epochs are required before the generated text starts sounding coherent.

It is recommended to run this script on GPU, as recurrent networks are quite computationally intensive.

If you try this script on new data, make sure your corpus has at least ~100k characters. ~1M is better.

In [1]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
from keras.utils import Sequence
from keras.callbacks import EarlyStopping
import numpy as np
import random
import io

Using TensorFlow backend.


In [0]:
class DataGenerator(Sequence):
    def __init__(self, text, char_indices, batch_size=128, maxlen=40, step=3):
        self.text = text
        self.char_indices = char_indices
        self.batch_size = batch_size
        self.maxlen = maxlen
        self.step = step

    def __len__(self):
        return ((len(self.text) - self.maxlen) // self.step) // self.batch_size

    def __getitem__(self, index):
        x = np.zeros((self.batch_size, self.maxlen, len(self.char_indices)), dtype=np.bool)
        y = np.zeros((self.batch_size, len(self.char_indices)), dtype=np.bool)

        for i in range(self.batch_size):
            idx = (i + index) * self.step

            for t, char in enumerate(self.text[idx: idx + self.maxlen]):
                x[i, t, self.char_indices[char]] = 1

            y[i, self.char_indices[self.text[idx + self.maxlen]]] = 1

        return x, y

In [3]:
path = get_file(
    'lovecraft.txt',
    origin='https://bashkirtsevich.pro/shared/lovecraft.txt'  # https://s3.amazonaws.com/text-datasets/nietzsche.txt
)

with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()

print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))

char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

23977984/23972914 [==============================] - 25s 1us/step
corpus length: 13137488
total chars: 123


In [0]:
maxlen = 40  # cut the text in semi-redundant sequences of maxlen characters
training_generator = DataGenerator(text, char_indices, maxlen=maxlen)

## Build the model: a single LSTM

In [0]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, _):
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

        print(generated)

In [6]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Instructions for updating:
Colocations handled automatically by placer.


In [7]:
model.fit_generator(
    generator=training_generator,
    validation_data=training_generator,
    epochs=60,
    callbacks=[
        LambdaCallback(on_epoch_end=on_epoch_end),
        EarlyStopping(monitor="loss", min_delta=0.001, patience=3, mode="min")
    ],
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/60
34212/34212 [==============================] - 4222s 123ms/step - loss: 0.8576 - val_loss: 0.4589
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: " в своем воображении я часто рисовал нев"
 в своем воображении я часто рисовал невилание о нетиском было выстим светом было были слуги последнего ли большего праграми, не смедла деревой немубысь у верл не обреть берего, который сневших потоковить ящико попротевеся к ним к можно в города и даворидом, у эте. что последнего медленные негом, подожде в темное оббывитые всему злогов, моь себе не отеником стене, достойм, что встомы межала, когда он приять на глазили камонны люд, после
----- diversity: 0.5
----- Generating with seed: " в своем воображении я часто рисовал нев"


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


 в своем воображении я часто рисовал невилан, что холминая сил, когда еще свое столько течтожили обнури над днем самом прокинето лео, слова, покостился ему времено и поляпыванное подграбе в темное сообнной стерами младного городовогоненсими и поятил попроясному себе не открате лесь. постарался в силиние в родазанием людом, что я последную легенди рез постично старанности, последню. не предолгеней. из лерапиным неуверхвались к течительно
----- diversity: 1.0
----- Generating with seed: " в своем воображении я часто рисовал нев"
 в своем воображении я часто рисовал невилание склепе, развый клясный клемы, в черно разорченадной лишного подобно подрял я больше. толеко последную лье, постирался катый датер в сила темной деревь постарался корабль от стлатие не покладил с крикной камне тромом и заплечищие и из своих прежде словно надвершеней гамоть к тогу, :д ожинь обширал его у нигалось катибля постраатьсящугом прописиного убровозом, тремивому же в лее встоку разост
----- diversity: 1.2
-----

KeyboardInterrupt: ignored